In [1]:
! pip install --quiet environs cyksuid toolz psycopg2-binary typing_json backoff pyyaml facebook_business pystache

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [ ]:
! pip install git+https://github.com/nandanrao/typedjson-python

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from environs import Env

from adopt.malaria import get_confs, get_df, load_basics, window, get_db_conf
from adopt.marketing import make_audience_conf, dict_from_nested_type
from adopt.marketing import Marketing
from adopt.facebook.state import CampaignState, get_api
from adopt.malaria import load_typed_json
from adopt.marketing import AudienceConf, CampaignConf
from adopt.campaign_queries import get_campaigns_for_user, create_campaign_confs, create_campaign_for_user
from typing import List
import json

env = Env()
env.read_env('.env-vlab-vlab', override=True)
db_conf = get_db_conf(env)

In [3]:

AD_CAMPAIGN = 'vaccine_vlab_wbg'
AD_ACCOUNT = '246148586918843'
PAGE_ID = '103760978380374'
RESPONDENT_AUDIENCE = "vlab-embed-wbg-vaccination-respondents"
INITIAL_SHORTCODE = 'vaccwbg'
SURVEY_SHORTCODES = ['vaccar', 'vaccen']
FINISHED_QUESTION_REF = 'age_cat'
EXTRA_METADATA = {'country': 'wbg'}

# Template adsets and strata creation

In [5]:
TEMPLATE_CAMPAIGNID = next(c['id'] for c in 
                           get_campaigns_for_user("mchatila@worldbank.org", db_conf) 
                           if c['name'] == "vaccination-iraq")

userinfo, config, db_conf, state, m, confs = load_basics(TEMPLATE_CAMPAIGNID, env)

In [6]:
template_state = CampaignState(userinfo.token, 
                               get_api(env, userinfo.token), 
                               '282370023153169', 
                               'embed-mena-wbg-templates')

In [7]:
import pandas as pd

share_lookup = pd.read_csv('outs/wbg-distribution-education.csv', header=[0,1,2], index_col=[0])

In [8]:
share = share_lookup.T.reset_index().melt(id_vars=['education', 'gender', 'age'], 
                                          var_name='location', 
                                          value_name='percentage')


In [9]:
from mena.strata import get_adsets, extraction_confs, hyphen_case

a, g, l = get_adsets(template_state, extraction_confs)    

variables = [
    { "name": "age", "source": "facebook", "conf": a},
    { "name": "gender", "source": "facebook", "conf": g},
    { "name": "location", "source": "facebook", "conf": l},
    { "name": "education", "source": "survey", "conf": {
        "levels": [{"name": "low", 
                    "audiences": [],
                    "excluded_audiences": [ "vlab-vacc-wbg-education-tertiary-lookalike", "vlab-vacc-wbg-education-secondary-lookalike"], 
                    "question_targeting": {"op": "or", 
                       "vars": [
                           {"op": "equal", "vars": [
                               {"type": "response", "value": "education"},
                               {"type": "constant", "value": "A"}]}, 
                           {"op": "equal", "vars": [
                               {"type": "response", "value": "education"},
                               {"type": "constant", "value": "B"}]}
                       ]}},
                   {"name": "secondary", 
                    "audiences": [], 
                    "excluded_audiences": ["vlab-vacc-wbg-education-low-lookalike", 
                                           "vlab-vacc-wbg-education-tertiary-lookalike"], 
                    "question_targeting": {"op": "equal", 
                                           "vars": [
                                               {"type": "response", "value": "education"},
                                               {"type": "constant", "value": "C"}
                                           ]}}, 
                   {"name": "tertiary", 
                    "audiences": ["vlab-vacc-wbg-education-tertiary-lookalike"], 
                    "excluded_audiences": [],
                    "question_targeting": {"op": "equal", 
                                           "vars": [
                                               {"type": "response", "value": "education"},
                                               {"type": "constant", "value": "D"}
                                           ]}}
                   ]}}
]

INFO:root:Loaded 8 adsets from campaign embed-mena-wbg-templates


In [10]:
def format_group_product(group, share_lookup):
    facebook_targeting = {}
    tvars = []
    md = {}
    conf = {"audiences": [], "excluded_audiences": []}

    id_list = []
    stratumid_list = []
    names = []

    for name, source, c in group:
        names += [c["name"]]
        id_list += [name, c["name"]]

        if source == "facebook":
            stratumid_list += [c["name"]]
            md_name = f"stratum_{name}"
            facebook_targeting = {**facebook_targeting, **c["params"]}
            md = {**md, md_name: c["name"]}

        if source == "survey":
            tvars.append(c["question_targeting"])
            conf["audiences"] += c["audiences"]
            conf["excluded_audiences"] += c["excluded_audiences"]

    variables = [name for name, _, _ in group]
    try:
        share = (
            share_lookup[variables + ["percentage"]]
            .set_index(variables)
            .loc[tuple(names)][0].item()
        )
    except KeyError as e:
        raise Exception(f"Could not find share for stratum: {names}") from e

    stratumid = "-".join([hyphen_case(s) for s in stratumid_list])
    md = {**md, "stratumid": stratumid}

    tvars.append({"op": "equal", "vars": [{"type": "response", "value": "md:stratumid"}, 
                                          {"type": "constant", "value": stratumid}]})

    tvars.append({ 'op': 'answered',
                   'vars': [{'type': 'response', 'value': FINISHED_QUESTION_REF}]})

    id_ = "-".join([hyphen_case(s) for s in id_list])

    conf = {
        "facebook_targeting": facebook_targeting,
        "question_targeting": {"op": "and", "vars": tvars},
        "metadata": md,
        **conf,
    }

    return id_, share, conf

In [11]:
from itertools import product 

groups = product(*[[(v['name'], v['source'], l) for l in v['conf']['levels']] for v in variables])
groups = [format_group_product(g, share) for g in groups]

# CREATING CONFS - WBG

In [12]:
USER = "dpinzonhernandez@worldbank.org"
CAMPAIGN = "vaccination-wbg"
CAMPAIGNID = next(c['id'] for c in get_campaigns_for_user(USER, db_conf) 
                  if c['name'] == CAMPAIGN)

# create_campaign_for_user(USER, CAMPAIGN, db_conf)

In [13]:
userinfo, config, db_conf, state, m, confs = load_basics(CAMPAIGNID, env)

In [14]:
images = {i['name']: i for i in state.account.get_ad_images(fields=['name', 'hash'])}

In [15]:
from adopt.facebook.update import Instruction
from adopt.malaria import run_instructions

def create_campaign(name):
    params = {
        "name": name,
        "objective": "MESSAGES",
        "status": "PAUSED",
        "special_ad_categories": [],
    }

    return Instruction("campaign", "create", params)


# run_instructions([create_campaign("vaccine_vlab_wbg")], state)

In [16]:
run_instructions([Instruction("campaign", "update", {"status": "ACTIVE"}, "23846641567840784")], state)

INFO:root:{'timestamp': '2021-03-02T15:32:40.195861+00:00', 'instruction': {'node': 'campaign', 'action': 'update', 'id': '23846641567840784', 'params': {'status': 'ACTIVE'}}}


In [ ]:
c = {'optimization_goal': 'LINK_CLICKS',
     'destination_type': 'MESSENGER',
     'adset_hours': 48,
     'budget': 300000.0,
     'min_budget': 100.0,
     'opt_window': 4*24,
     'end_date': '2021-03-21',
     'proportional': True,
     'page_id': PAGE_ID,
     'instagram_id': None,
     'ad_account': AD_ACCOUNT,
     'ad_campaign': AD_CAMPAIGN}

config = CampaignConf(**c)

create_campaign_confs(CAMPAIGNID, "opt", [config._asdict()], db_conf)

In [ ]:
from adopt.marketing import make_audience_conf, dict_from_nested_type
import json

audiences = [
    {
        "name": "vlab-vacc-wbg-education-low",
        "shortcodes": SURVEY_SHORTCODES,
        "subtype": "LOOKALIKE",
        "lookalike": {
            "name": "vlab-vacc-wbg-education-low-lookalike",
            "target": 500,
            "spec": {
                "country": "PS",
                "starting_ratio": 0.0,
                "ratio": 0.2
            }
        },
        "question_targeting": {"op": "or", 
                               "vars": [
                                   {"op": "equal", "vars": [
                                       {"type": "response", "value": "education"},
                                       {"type": "constant", "value": "A"}]}, 
                                   {"op": "equal", "vars": [
                                       {"type": "response", "value": "education"},
                                       {"type": "constant", "value": "B"}]}
                               ]}
    },
    {
        "name": "vlab-vacc-wbg-education-secondary",
        "shortcodes": SURVEY_SHORTCODES,
        "subtype": "LOOKALIKE",
        "lookalike": {
            "name": "vlab-vacc-wbg-education-secondary-lookalike",
            "target": 2000,
            "spec": {
                "country": "PS",
                "starting_ratio": 0.0,
                "ratio": 0.2
            }
        },
        "question_targeting": {"op": "equal", 
                               "vars": [
                                   {"type": "response", "value": "education"},
                                   {"type": "constant", "value": "C"}
                               ]}

    },
    {
        "name": "vlab-vacc-wbg-education-tertiary",
        "shortcodes": SURVEY_SHORTCODES,
        "subtype": "LOOKALIKE",
        "lookalike": {
            "name": "vlab-vacc-wbg-education-tertiary-lookalike",
            "target": 2000,
            "spec": {
                "country": "PS",
                "starting_ratio": 0.0,
                "ratio": 0.2
            }
        },
        "question_targeting": {"op": "equal", 
                               "vars": [
                                   {"type": "response", "value": "education"},
                                   {"type": "constant", "value": "D"}
                               ]}

    },
    {
        "name": RESPONDENT_AUDIENCE,
        "shortcodes": [INITIAL_SHORTCODE],
        "subtype": "CUSTOM"
    },
]


audience_confs = [make_audience_conf(c) for c in audiences]
confs = [dict_from_nested_type(a) for a in audience_confs]

create_campaign_confs(CAMPAIGNID, "audience", confs, db_conf)

In [16]:
from adopt.marketing import CreativeConf


def _creative_conf(name, image, body, headline, welcome_message, button_text, form):
    return {
        "name": name,
        "image": image['name'],
        "image_hash": image['hash'],
        "body": body,
        "link_text": headline,
        "welcome_message": welcome_message,
        "button_text": button_text,
        "form": form,
    }


df = pd.read_csv('outs/creative-wbg.csv')
image_confs = [{**d, 'image': images[d['image']]} for d in df.to_dict(orient='records')]

creatives = [CreativeConf(**_creative_conf(**{**c, 'form': INITIAL_SHORTCODE})) for c in image_confs]
confs = [c._asdict() for c in creatives]

create_campaign_confs(CAMPAIGNID, "creative", confs, db_conf)

In [17]:
ALL_CREATIVES = [t['name'] for t in image_confs]

def make_stratum(id_, quota, c):
    return { 'id': id_,
             'metadata': {**c['metadata'], **EXTRA_METADATA},
             'facebook_targeting': c['facebook_targeting'], 
             'creatives': ALL_CREATIVES,
             'audiences': c['audiences'],
             'excluded_audiences': [*c['excluded_audiences'], RESPONDENT_AUDIENCE],
             'quota': quota,
             'shortcodes': SURVEY_SHORTCODES,
             'question_targeting': c['question_targeting']}

In [ ]:
from adopt.marketing import make_stratum_conf, StratumConf, QuestionTargeting

import typedjson
strata = [make_stratum(*g) for g in groups]
strata_data = [dict_from_nested_type(make_stratum_conf(c)) for c in strata]
create_campaign_confs(CAMPAIGNID, "stratum", strata_data, db_conf)

In [254]:
sc = make_stratum_conf(strata[0])

In [ ]:
dir(sc.facebook_targeting['geo_locations'])

# TESTING

In [98]:
mal = load_basics(CAMPAIGNID, env)

In [6]:
%%time

from adopt.malaria import update_ads_for_campaign, update_audience_for_campaign
 
# instructions, report = update_ads_for_campaign(mal)

instructions, report = update_audience_for_campaign(mal)

CPU times: user 1min 23s, sys: 113 ms, total: 1min 23s
Wall time: 1min 41s


In [99]:
from adopt.malaria import hydrate_strata

userinfo, config, db_conf, state, m, confs = mal
strata = hydrate_strata(state, confs["stratum"], confs["creative"])

df = get_df(db_conf, userinfo.survey_user, strata)

In [104]:
from adopt.malaria import hydrate_audiences

userinfo, config, db_conf, state, m, confs = mal
audience_confs = confs["audience"]

df = get_df(db_conf, userinfo.survey_user, audience_confs)

audiences = hydrate_audiences(df, m, audience_confs)

In [14]:
from adopt.responses import get_surveyids

surveyids = get_surveyids({'vaccwbg'}, userinfo.survey_user, db_conf)

In [ ]:
from adopt.campaign_queries import create_adopt_report

db_conf['user'] = 'adopt'
create_adopt_report(CAMPAIGNID, 'FACEBOOK_ADOPT', report, db_conf)

In [ ]:
from adopt.malaria import run_instructions

run_instructions(instructions, mal.state)

In [70]:
import pandas as pd
from adopt.campaign_queries import get_last_adopt_report

rdf = pd.DataFrame(get_last_adopt_report(CAMPAIGNID, "FACEBOOK_ADOPT", mal.db_conf)).T

In [ ]:
rdf.sort_values('respondents')

In [ ]:
mal.state.custom_audiences

In [ ]:
mal.state.spend

In [ ]:
# What's going on with spend? 
# spend should come...Maybe the lookalike audiences weren't in effect

rdf.sort_values('respondents')

In [84]:
import json
from facebook_business.adobjects.application import Application

def make_event(pageid, userid):
    return {
        'event': 'CUSTOM_APP_EVENTS',
        'custom_events': json.dumps([{
            '_eventName': "survey_completion",
        }]),
        'advertiser_tracking_enabled': 1,
        'application_tracking_enabled': 1,
        'extinfo': json.dumps(['mb1']),
        'page_id': pageid,
        'page_scoped_user_id': userid,
    }


app_id = env("FACEBOOK_APP_ID")
app = Application(fbid=app_id, api=mal.state.api)

In [93]:
pageid = mal.confs['opt'][0].page_id
params = make_event(pageid, audiences[3].users[0])
app.create_activity(params=params)

'2301506143306723'